In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [2]:
# preprocessing
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),# random crop and resized
    transforms.RandomRotation(20), # random rotation
    transforms.RandomHorizontalFlip(p=0.5), # random horizontal flip
    transforms.ToTensor() 
])

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

print(device)

# read data
root = 'Pin'
train_dataset = datasets.ImageFolder(root + '/train', transform)
test_dataset = datasets.ImageFolder(root + '/test', transform)

# load data
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)

cuda


In [3]:
classes = train_dataset.classes
classes_index = train_dataset.class_to_idx
print(classes)
print(classes_index)

['md2', 'morris']
{'md2': 0, 'morris': 1}


In [4]:
model = models.vgg16(pretrained = True)
print(model)

C:\Users\hwi\anaconda3\envs\torch_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hwi\anaconda3\envs\torch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
# construct Fully Connected Layer
model.classifier = torch.nn.Sequential(torch.nn.Linear(25088, 100),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(100, 2))

In [6]:
LR = 0.0001
# Cost Function 
entropy_loss = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.SGD(model.parameters(), LR, momentum=0.9)

In [7]:
def train():
    model.train()
    for i, data in enumerate(train_loader):
        # get the data and labels
        inputs, labels = data
        # Get model prediction results, (64，10）
        out = model(inputs)
        # Cross-entropy cost function out(batch,C),labels(batch)
        loss = entropy_loss(out, labels)
        # zero grad
        optimizer.zero_grad()
        # compute grad
        loss.backward()
        # edit weight
        optimizer.step()


def test():
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # get the data and labels
        inputs, labels = data
        # Get model prediction results, (64，10）
        out = model(inputs)
        # Cross-entropy cost function out(batch,C),labels(batch)
        _, predicted = torch.max(out, 1)
        # correct amount
        correct += (predicted == labels).sum()
    print("Test acc: {0}".format(correct.item() / len(test_dataset)))
    
    correct = 0
    for i, data in enumerate(train_loader):
        # get the data and labels
        inputs, labels = data
        # Get model prediction results
        out = model(inputs)
        # get max, and max position
        _, predicted = torch.max(out, 1)
        # correct amount
        correct += (predicted == labels).sum()
    print("Train acc: {0}".format(correct.item() / len(train_dataset)))

In [8]:
for epoch in range(0, 10):
    print('epoch:',epoch)
    train()
    test()
    
torch.save(model.state_dict(), 'morris_md2.pth')

epoch: 0
Test acc: 0.8404907975460123
Train acc: 0.8969072164948454
epoch: 1
Test acc: 0.8466257668711656
Train acc: 0.9097938144329897
epoch: 2
Test acc: 0.901840490797546
Train acc: 0.9510309278350515
epoch: 3
Test acc: 0.9263803680981595
Train acc: 0.9729381443298969
epoch: 4
Test acc: 0.9325153374233128
Train acc: 0.9716494845360825
epoch: 5
Test acc: 0.950920245398773
Train acc: 0.9780927835051546
epoch: 6
Test acc: 0.9447852760736196
Train acc: 0.9832474226804123
epoch: 7
Test acc: 0.9693251533742331
Train acc: 0.9845360824742269
epoch: 8
Test acc: 0.9570552147239264
Train acc: 0.979381443298969
epoch: 9
Test acc: 0.9693251533742331
Train acc: 0.9935567010309279
